# Article Data Features
Example code snippets for calculating various standard features related to Wikipedia articles

## Setup

In [2]:
import bz2
from datetime import datetime, timedelta
import os
import re
import sys

import mwparserfromhell
import numpy as np
import pandas as pd
import wmfdata

In [3]:
spark = wmfdata.spark.get_session(app_name='pyspark reg; article-data-exploration',
                                  type='yarn-regular', # local, yarn-regular, yarn-large
                                  )  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [4]:
print("Current mediawiki snapshot partitions:")
spark.sql("SHOW PARTITIONS wmf_raw.mediawiki_project_namespace_map").show(50, False)

print("\nCurrent wikidata snapshot partitions:")
spark.sql("SHOW PARTITIONS wmf.wikidata_item_page_link").show(50, False)

Current mediawiki snapshot partitions:
+------------------------+
|partition               |
+------------------------+
|snapshot=2016-12_private|
|snapshot=2017-07_private|
|snapshot=2021-04        |
|snapshot=2021-05        |
|snapshot=2021-06        |
|snapshot=2021-07        |
|snapshot=2021-08        |
|snapshot=2021-09        |
+------------------------+


Current wikidata snapshot partitions:
+-------------------+
|partition          |
+-------------------+
|snapshot=2021-08-30|
|snapshot=2021-09-06|
|snapshot=2021-09-13|
|snapshot=2021-09-20|
|snapshot=2021-09-27|
|snapshot=2021-10-04|
+-------------------+



In [5]:
mediawiki_snapshot = '2021-09'  # data will be current to this month inclusive -- e.g., 2020-07 means data is up to 31 July 2020 (at least)
wikidata_snapshot = '2021-10-04'  # data will be current to approximately this day

## Utilities
Various random utility functions that can be useful

In [ ]:
def getNamespacePrefixes(lang):
    """Get list of namespace prefixes for a given wiki.
    
    This should be run in a standard Python cell and then the output
    fed into a call to filterLinksByNs as part of a SQL query.
    """
    session = requests.Session()
    base_url = "https://{0}.wikipedia.org/w/api.php".format(lang)
    params = {
        "action": "query",
        "meta": "siteinfo",
        "siprop": "namespacealiases|namespaces",
        "format": "json",
        "formatversion": "2"
    }
    result = session.get(url=base_url, params=params)
    result = result.json()
    prefix_to_ns = {}
    if 'namespacealiases' in result.get('query', {}):
        for alias in result['query']['namespacealiases']:
            prefix_to_ns[alias['alias']] = alias['id']
    if 'namespaces' in result.get('query', {}):
        for ns in result['query']['namespaces'].values():
            if 'name' in ns:
                prefix_to_ns[ns['name'].replace(' ', '_')] = ns['id']
            if 'canonical' in ns:
                prefix_to_ns[ns['canonical'].replace(' ', '_')] = ns['id']
    return prefix_to_ns

def filterLinksByNs(links, keep_ns):
    """Filter down a list of link targets (page titles) by namespace -- e.g., keep_ns=6 for just File links."""
    for i in range(len(links)-1, -1, -1):
        link_ns = 0
        if ':' in links[i]:
            prefix = links[i].split(':')[0].replace(' ', '_')
            if prefix in namespace_prefixes:
                link_ns = namespace_prefixes[prefix]
        if link_ns not in keep_ns:
            links.pop(i)
    return links

spark.udf.register('filterLinksByNs', filterLinksByNs, 'Array<String>')

In [ ]:
def removeComments(wikitext):
    """Wikitext can have comments in it that should be removed before checking for links etc.
    
    In practice this has no effect on pretty much every article but some articles do have large comments
    in them with either context about the content or old wikitext that someone didn't want to actually delete.
    """
    return re.sub('<!--.*?-->', '', wikitext, flags=re.DOTALL)

spark.udf.register('removeComments', removeComments, 'String')

## Wikitext Attributes
These functions should all be called with code like:

`WITH wikitext AS (
    SELECT
      revision_text
    FROM wmf.mediawiki_wikitext_current
    WHERE
      snapshot = '{mediawiki_snapshot}'
      AND ...
)
SELECT
  LENGTH(getLinks(revision_text)) AS num_links,
  getNumRefs(revision_text) AS num_refereneces,
  ...
FROM wikitext`

NOTE: for most of these functions, there is generally a `mwparserfromhell` version and regex version. The regex versions are pretty good replacements and in theory faster / easier to deploy to workers because of their simplicity but do not catch as many edge-cases as `mwparserfromhell` does so if you have the luxury of using `mwparserfromhell`, go with it.

In [4]:
def getLinksMWP(wikitext):
    """Extract list of links from wikitext for an article via mwparserfromhell.
    
    This data is much more easily extracted via the pagelinks table but this wikitext-based approach
    is important if you're trying to filter out links added via templates or need context about where
    links are found on a page.
    
    NOTE: this approache removes the section anchors from links so it can be more easily matched against
    other data about page titles but you could remove the `.partition('#')[0]` part to retain them.
    """
    try:
        wt = mwparserfromhell.parse(wikitext)
        return [str(l.title).partition('#')[0].replace(' ', '_').strip() for l in wt.filter_wikilinks()]
    except Exception:
        return None
    
def getLinksRegex(wikitext):
    """Extract list of links from wikitext for an article via simple regex.
    
    Known Issues:
    * Doesn't handle nested wikilinks (just gets the first) -- e.g., [[File:filename|caption=[[second_wikilink]]]] would be just File:filename
    """
    try:
        return [m.split('|')[0].partition('#')[0].replace(' ', '_').strip() for m in re.findall('(?<=\[\[)(.*?)(?=\]\])', wikitext, flags=re.DOTALL)]
    except Exception:
        return None

    
spark.udf.register('getLinks', getLinksMWP, 'Array<String>')

<function __main__.getLinksMWP(wikitext)>

In [5]:
sfn_templates = ["Shortened footnote template", "sfn", "Sfnp", "Sfnm", "Sfnmp"]

def getNumRefsMWP(wikitext):
    """Extract a count of references from wikitext for an article via mwparserfromhell.
    
    NOTE: the shortened footnote templates are used in some wikis but <ref> tags are definitely the standard
    and it would be fair to just remove that portion of the code.
    """
    try:
        wt = mwparserfromhell.parse(wikitext)
        num_ref_tags = len([t.tag for t in wt.filter_tags() if t.tag == 'ref'])
        num_sftn_templates = len([t.name for t in mwparserfromhell.parse(wikitext).filter_templates() if t.name in sfn_templates])
        return num_ref_tags + num_sftn_templates
    except Exception:
        return None
    
def getNumRefsRegex(wikitext):
    """Regex version for extracting references.
    
    Not implemented here but see for potential code:
    https://github.com/mediawiki-utilities/python-mwrefs/blob/master/mwrefs/references/extract.py
    """
    pass
    
spark.udf.register('getNumRefs', getNumRefsMWP, 'Int')

<function __main__.getRefsMWP(wikitext)>

In [7]:
def getHeadingsMWP(wikitext, level=None):
    """Extract list of headings from wikitext for an article via mwparserfromhell."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        if level is None:
            return [str(l.title).strip() for l in wt.filter_headings()]
        else:
            return [str(l.title).strip() for l in wt.filter_headings() if l.level == level]
    except Exception:
        return None
    
def getHeadingsRegex(wikitext, level=None):
    """Extract list of headings from wikitext for an article via simple regex."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        if level is None:
            return [l[1].strip() for l in re.findall('(={2,})(.*?)(={2,})', wikitext)]
        else:
            return [l[1].strip() for l in re.findall('(={2,})(.*?)(={2,})', wikitext) if len(l[0]) == level]
    except Exception:
        return None
    
spark.udf.register('getHeadings', getHeadingsMWP, 'Array<String>')

<function __main__.getHeadingsMWP(wikitext, level=None)>

In [8]:
def getTemplatesMWP(wikitext):
    """Extract list of templates from wikitext for an article via mwparserfromhell."""
    try:
        wt = mwparserfromhell.parse(wikitext)
        return [str(t.name).replace(' ', '_').strip() for t in wt.filter_templates()]
    except Exception:
        return None
    
def getTemplatesRegex(wikitext):
    """Extract list of templates from wikitext for an article via simple regex.

    Known Issues:
    * Doesn't handle nested templates (just gets the first)
    -- e.g., '{{cite web|url=http://www.allmusic.com/|ref={{harvid|AllMusic}}}}' would be just web
    """
    try:
        return [m.split('|')[0].replace(' ', '_').strip() for m in re.findall('(?<=\{\{)(.*?)(?=\}\})', wikitext, flags=re.DOTALL)]
    except Exception:
        return None
    
spark.udf.register('getTemplates', getTemplatesMWP, 'Array<String>')

<function __main__.getTemplatesMWP(wikitext)>

## Mediawiki Table Features

### Pagelinks
This query is targeted at Wikipedia and resolves redirects, maps page titles to page IDs and Wikidata IDs. It could be easily adapted to other namespaces, wikis, etc.

CTEs:
* `wikipedia_projects`: get wiki_dbs for all Wikipedia language editions -- e.g., `enwiki` for English Wikipedia
* `title_to_id`: mapping of page ID to page title for each wiki
* `redirects`: table of redirects in source page ID -> target page ID format
* `pagelinks_reformatted`: pagelinks in source page ID -> target page ID format and with redirect pages removed (when they're the source of the link).
* `pagelinks_redirects_resolved`: when target page ID is a redirect, resolve it. The `DISTINCT` clause ensures that no link pair occurs more than once (can happen when resolving redirects)
* `wikidata_ids`: build table of page ID -> wikidata ID
* `SELECT...`: join in Wikidata IDs to page links

From this table, it would be easy to compute outlinks or inlinks:
* `outlinks`: `SELECT wiki_db, pid_from, count(pid_to) AS num_outlinks FROM <links-table> GROUP BY wiki_db, pid_from`
* `inlinks`: `SELECT wiki_db, pid_to, count(pid_from) AS num_inlinks FROM <links-table> GROUP BY wiki_db, pid_to`

In [ ]:
query = f"""
WITH wikipedia_projects AS (
        SELECT DISTINCT dbname
          FROM wmf_raw.mediawiki_project_namespace_map
         WHERE snapshot = '{mediawiki_snapshot}'
               AND hostname LIKE '%wikipedia%'
        ),
title_to_id AS (
    SELECT page_id,
           page_title,
           wiki_db
      FROM wmf_raw.mediawiki_page mp
     INNER JOIN wikipedia_projects wp
           ON (mp.wiki_db = wp.dbname)
     WHERE page_namespace = 0
           AND snapshot = '{mediawiki_snapshot}'
),
redirects AS (
    SELECT mr.rd_from AS rd_from,
           tti.page_id AS rd_to,
           mr.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_redirect mr
     INNER JOIN title_to_id tti
           ON (mr.rd_title = tti.page_title
               AND mr.wiki_db = tti.wiki_db)
     WHERE mr.snapshot = '{mediawiki_snapshot}'
           AND mr.rd_namespace = 0
),
pagelinks_reformatted AS (
    SELECT pl.pl_from AS pl_from,
           tti.page_id AS pl_to,
           pl.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_pagelinks pl
     INNER JOIN title_to_id tti
           ON (pl.pl_title = tti.page_title
               AND pl.wiki_db = tti.wiki_db)
      LEFT ANTI JOIN redirects r
           ON (pl.pl_from = r.rd_from
               AND pl.wiki_db = r.wiki_db)
     WHERE snapshot = '{mediawiki_snapshot}'
           AND pl_from_namespace = 0
           AND pl_namespace = 0
),
pagelinks_redirects_resolved AS (
    SELECT DISTINCT pl.pl_from AS pl_from,
           COALESCE(r.rd_to, pl.pl_to) AS pl_to,
           pl.wiki_db AS wiki_db
      FROM pagelinks_reformatted pl
      LEFT JOIN redirects r
           ON (pl.pl_to = r.rd_from
               AND pl.wiki_db = r.wiki_db)
),
wikidata_ids AS (
    SELECT DISTINCT wiki_db,
           page_id,
           item_id
      FROM wmf.wikidata_item_page_link wd
     INNER JOIN wikipedia_projects p
           ON (wd.wiki_db = p.dbname)
     WHERE wd.snapshot = '{wikidata_snapshot}'
           AND wd.page_namespace = 0
)
SELECT wf.item_id AS qid_from,
       p.pl_from AS pid_from,
       wt.item_id AS qid_to,
       p.pl_to AS pid_to,
       p.wiki_db as wiki_db
  FROM pagelinks_redirects_resolved p
  LEFT JOIN wikidata_ids wf
       ON (p.pl_from = wf.page_id
           AND p.wiki_db = wf.wiki_db)
  LEFT JOIN wikidata_ids wt
       ON (p.pl_to = wt.page_id
           AND p.wiki_db = wt.wiki_db)
"""

result = spark.sql(query)

### Category links
This is untested but my quick attempt at porting the pagelinks code to categories:

Potential issues:
* Do article redirect pages ever have categories? Should they be excluded explicitly?

CTEs:
* `wikipedia_projects`: get wiki_dbs for all Wikipedia language editions -- e.g., `enwiki` for English Wikipedia
* `category_title_to_id`: mapping of page ID to page title for each wiki for categories
* `article_title_to_id`: same but for articles
* `redirects`: table of redirects in source page ID -> target page ID format. This is for categories.
* `categorylinks_reformatted`: categorylinks in article page ID -> target category page ID format. 
* `categorylinks_redirects_resolved`: when target category is a redirect, resolve it. The `DISTINCT` clause ensures that no link pair occurs more than once (though I think in theory isn't necessary...).
* `wikidata_ids`: build table of page ID -> wikidata ID for articles and categories
* `SELECT...`: join in Wikidata IDs to category links

In [ ]:
query = f"""
WITH wikipedia_projects AS (
        SELECT DISTINCT dbname
          FROM wmf_raw.mediawiki_project_namespace_map
         WHERE snapshot = '{mediawiki_snapshot}'
               AND hostname LIKE '%wikipedia%'
        ),
category_title_to_id AS (
    SELECT page_id,
           page_title,
           wiki_db
      FROM wmf_raw.mediawiki_page mp
     INNER JOIN wikipedia_projects wp
           ON (mp.wiki_db = wp.dbname)
     WHERE page_namespace = 14
           AND snapshot = '{mediawiki_snapshot}'
),
article_title_to_id AS (
    SELECT page_id,
           page_title,
           wiki_db
      FROM wmf_raw.mediawiki_page mp
     INNER JOIN wikipedia_projects wp
           ON (mp.wiki_db = wp.dbname)
     WHERE page_namespace = 0
           AND snapshot = '{mediawiki_snapshot}'
),
redirects AS (
    SELECT r.rd_from AS rd_from,
           cat.page_id AS rd_to,
           r.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_redirect r
     INNER JOIN category_title_to_id cat
           ON (r.rd_title = cat.page_title
               AND r.wiki_db = cat.wiki_db)
     WHERE mr.snapshot = '{mediawiki_snapshot}'
           AND mr.rd_namespace = 14
),
categorylinks_reformatted AS (
    SELECT cl.cl_from AS art_from,
           cat.page_id AS cat_to,
           cl.wiki_db AS wiki_db
      FROM wmf_raw.mediawiki_categorylinks cl
     INNER JOIN article_title_to_id art
           ON (cl.cl_from = art.page_id
               AND cl.wiki_db = art.wiki_db)
     INNER JOIN category_title_to_id cat
           ON (cl.pl_title = cat.page_title
               AND cl.wiki_db = cat.wiki_db)
     WHERE snapshot = '{mediawiki_snapshot}'
),
categorylinks_redirects_resolved AS (
    SELECT DISTINCT cl.art_from AS art_from,
           COALESCE(r.rd_to, cl.cat_to) AS cat_to,
           cl.wiki_db AS wiki_db
      FROM categorylinks_reformatted cl
      LEFT JOIN redirects r
           ON (cl.pl_to = r.rd_from
               AND cl.wiki_db = r.wiki_db)
),
wikidata_ids AS (
    SELECT DISTINCT wiki_db,
           page_id,
           item_id
      FROM wmf.wikidata_item_page_link wd
     INNER JOIN wikipedia_projects p
           ON (wd.wiki_db = p.dbname)
     WHERE wd.snapshot = '{wikidata_snapshot}'
           AND (wd.page_namespace = 14
                OR wd.page_namespace = 0)
)
SELECT wf.item_id AS qid_from,
       p.art_from AS art_from,
       wt.item_id AS qid_to,
       p.cat_to AS cat_to,
       p.wiki_db as wiki_db
  FROM categorylinks_redirects_resolved p
  LEFT JOIN wikidata_ids wf
       ON (p.art_from = wf.page_id
           AND p.wiki_db = wf.wiki_db)
  LEFT JOIN wikidata_ids wt
       ON (p.cat_to = wt.page_id
           AND p.wiki_db = wt.wiki_db)
"""

result = spark.sql(query)

### Image links

Probably verify with Miriam or Aiko as there might be things about images I'm unaware of -- e.g., are there image redirects that I should be accounting for?

CTEs:
* `wikipedia_projects`: get wiki_dbs for all Wikipedia language editions -- e.g., `enwiki` for English Wikipedia
* `pages`: set of all Wikipedia articles for gathering image data
* `SELECT...`: gather all imagelinks for all articles and count up.

In [ ]:
query = f"""
WITH wikipedia_projects AS (
    SELECT DISTINCT
      dbname
    FROM wmf_raw.mediawiki_project_namespace_map
    WHERE
      snapshot = '{mediawiki_snapshot}'
      AND hostname LIKE '%wikipedia%'
),
pages AS (
    SELECT
      wiki_db,
      page_id
    FROM wmf_raw.mediawiki_page p
    INNER JOIN wikipedia_projects wp
      ON (p.wiki_db = wp.dbname)
    WHERE
      snapshot = '{mediawiki_snapshot}'
      AND page_namespace = 0
      AND NOT page_is_redirect
)
SELECT
  i.wiki_db,
  il_from AS page_id,
  COUNT(DISTINCT(il_to)) AS num_images
FROM wmf_raw.mediawiki_imagelinks i
INNER JOIN pages p
  ON (i.il_from = p.page_id
      AND i.wiki_db = p.wiki_db)
WHERE
  snapshot = '{mediawiki_snapshot}'
  AND il_from_namespace = 0
GROUP BY
  i.wiki_db,
  il_from
"""

result = spark.sql(query)

## Equity Features
* Gender: https://github.com/geohci/miscellaneous-wikimedia/blob/master/wikidata-properties-spark/wikidata_gender_information.ipynb
* Geography: https://github.com/geohci/wiki-region-groundtruth/blob/main/wiki-region-data.ipynb

## Visibility Features

### Featured articles
This is sorta messy in that each wiki tracks featured articles differently. Could design a category-based approach but I think Wikidata badges are the way to go for a single universal source. Probably should do some evaluation to see if it captures everything we expect though.

CTEs:
* `wikipedia_projects`: get wiki_dbs for all Wikipedia language editions -- e.g., `enwiki` for English Wikipedia
* `relevant_qids`: set of all Wikidata items with Wikipedia articles
* `exploded_sitelinks`: for all relevant Wikidata items, get all sitelinks -- one row per sitelink (could retain the Wikidata `item_id` too here if desired). Each sitelink will have an associated page title for the Wikidata item and any information about badges.
* `all_badges`: further expand out badge information for any sitelinks that have them.
* `relevant_badges`: narrow down to just badges that indicate high-quality articles (Good, Featured, or Recommended)
* `relevant_pages`: get all Wikipedia articles
* `SELECT...`: keep badges that are associated with Wikipedia articles and join in page IDs

In [ ]:
# Good Article: Q17437798
# Featured Article: Q17437796
# Recommended Article: Q17559452

query = f"""
WITH wikipedia_projects AS (
    SELECT DISTINCT dbname
      FROM wmf_raw.mediawiki_project_namespace_map
     WHERE snapshot = '{mediawiki_snapshot}'
           AND hostname LIKE '%wikipedia%'
),
relevant_qids AS (
    SELECT DISTINCT item_id
      FROM wmf.wikidata_item_page_link wd
     INNER JOIN wikipedia_projects wp
           ON (wd.wiki_db = wp.dbname)
     WHERE snapshot = '{wikidata_snapshot}'
           AND page_namespace = 0
),
exploded_sitelinks AS (
    SELECT explode(sitelinks) as sitelink
      FROM wmf.wikidata_entity w
     INNER JOIN relevant_qids q
           ON (w.id = q.item_id)
     WHERE w.snapshot = '{wikidata_snapshot}'
),
all_badges AS (
    SELECT sitelink.site AS wiki_db,
           sitelink.title AS page_title,
           EXPLODE(sitelink.badges) AS badge
     FROM exploded_sitelinks
    WHERE size(sitelink.badges) > 0
),
relevant_badges AS (
    SELECT wiki_db,
           page_title
      FROM all_badges
     WHERE badge IN ('Q17437798', 'Q17437796', 'Q17559452')
),
relevant_pages AS (
    SELECT wiki_db,
           page_id,
           page_title
      FROM wmf_raw.mediawiki_page p
     INNER JOIN wikipedia_projects wp
           ON (p.wiki_db = wp.dbname)
     WHERE snapshot = '{mediawiki_snapshot}'
           AND page_namespace = 0
)
SELECT b.wiki_db,
       b.page_title,
       p.page_id
  FROM relevant_badges b
 INNER JOIN relevant_pages p
       ON (b.wiki_db = p.wiki_db
           AND b.page_title = p.page_title)
"""

result = spark.sql(query)